In [ ]:
import numpy as np
import gymnasium as gym
gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
import random
import time
from IPython.display import clear_output

In [ ]:
#new_step_api = True
env = gym.make('FrozenLake-v1', desc=generate_random_map(size=8))
new_step_api = True

In [ ]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size,action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
num_episodes = 10000 #total no of episodes our agent will play during the training period
max_steps_per_episode = 100 #max no of steps our agent can take in each episode

learning_rate = 0.1 #the amount of q value agent will take from the old q value for a specific (s,a) pair
discount_rate = 0.99 #gamma in bellman optimal equation

#exploration and exploitation tradeoffs
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [ ]:
rewards_all_episodes = [] #used to hold all the rewards in each episode

#Q-Learning algorithm
for episode in range(num_episodes): #this loop contains everything that happens within a single episode
  state,_ = env.reset() #set our env back to starting state for each episode, unpack the tuple to get state

  done = False #checks whether our episode is finished
  rewards_current_episode = 0 #stores the rewards within the current episode

  for step in range(max_steps_per_episode): #contains everything that happens at a single time stamp within each episode

    #Exploration-exploitation trade-off
    exploration_rate_threshold = random.uniform(0,1) #this value will determine if our agent will explore or exploit for this time step
    if exploration_rate_threshold > exploration_rate: #explor - exploit trade off logic
      action = np.argmax(q_table[state,:])   #this will return the max value in our current q values
    else:
      action = env.action_space.sample() #the agent will explore the env and sample an action randomnly

    #after our action is taken we then will call the action with .step, which will pass our action to env. .step will return a tuple which contains
    #the new state(new_state), the reward our agent got with that action, done(whether or not our action ended that episode or not) and info(which contains some more
    # diagnostic information regarding our environment which can later be helpful for us to debug)
    new_state, reward, terminated, truncated, info = env.step(action)

    #After analyzing reward which we got from our action, we will then update our q value for that (s,a) pair in the q table.
    #Update Q-table for Q(s,a)
    q_table[state,action] = q_table[state,action] * (1- learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))

    state = new_state #now we will update our current state with the new state which we got before
    rewards_current_episode += reward #we will also update our reward by adding it to the previous rewards(rewards_current_episode)

    if done == True: #this will check if our agent reached to its desired state in this episode
       break

  # Exploration rate decay (Once the episode is complete, we need to update our exploration rate by using exploration rate decay)
  exploration_rate = min_exploration_rate +  (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)

  rewards_all_episodes.append(rewards_current_episode) #append the reward from our current episode to the list which contains the rewards from all the episodes

#Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000
print("*******Average reward per thousand episodes*******\n")
for r in rewards_per_thousand_episodes:
  print(count, ": ", str(sum(r/1000)))
  count += 1000

*******Average reward per thousand episodes*******

1000 :  0.0
2000 :  0.0
3000 :  0.0
4000 :  0.0
5000 :  0.0
6000 :  0.0
7000 :  0.0
8000 :  0.0
9000 :  0.0
10000 :  0.0


In [ ]:
#print updated q table
print("\n\n *******Q-table********\n")
print(q_table)



 *******Q-table********

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
 #Our agent will now play the game after the previous learning

 for episode in range(3): #taking range as 3 means we will go through 3 episodes
     state,info = env.reset() #this will reset the environment from scratch for the next episode in loop, unpack the tuple
     done = False #will be used to prompt weather the agent has reached or not
     print("*****EPISODE ", episode + 1, "*****\n\n\n\n")
     time.sleep(1) #This is to give us time to read the output before it disappears from the screen.

     for step in range(max_steps_per_episode): #this loop is for each time step
         clear_output(wait=True) #this will clear the last output from the notebook cell for clarity. by setting wait as true, it waits for the program to clear before it writes another output.
         gym.make('FrozenLake-v1', desc=generate_random_map(size=8))#this will render the environment into the screen so that we can see what is happening
         time.sleep(1) #we will see the rendered env for 900ms on screen

         action = np.argmax(q_table[state,:]) #set action which has the highest q value for the current state
         new_state, reward, terminated, truncated, info = env.step(action) #as discussed above, action returns 3 values.
         done = terminated or truncated #check if the episode is done


         if done:
             clear_output(wait=True)
             gym.make('FrozenLake-v1', desc=generate_random_map(size=8))
             if reward == 1:
                 print("*****YOU REACHED THE GOAL!!")
                 time.sleep(3)
             else:
                 print("*****You fell into a hole!*****")
                 time.sleep(3)

             break # Break out of the loop when the episode is done

         state = new_state #After the episode is done, we will move our current state to the new state given by the action.

 env.close()


*****You fell into a hole!*****
